In [2]:
from selenium import webdriver
from time import sleep
import pandas as pd
from selenium.webdriver.common.action_chains import ActionChains  # 用于鼠标悬停
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

import matplotlib.pyplot as plt
# 如果需要 inline 显示
%matplotlib inline


In [ ]:

# 启动 Edge 浏览器
driver = webdriver.Edge()

# 访问天津二手房首页
url = 'https://tj.esf.fang.com/'
driver.get(url)

# 可视化效果时稍微等待下页面加载
sleep(2)


In [11]:
# 找到悬停的元素（s4Box）
hover_element = driver.find_element(By.CSS_SELECTOR, "div.s4Box")

# 创建动作对象
actions = ActionChains(driver)
# 将鼠标移动到目标元素
actions.move_to_element(hover_element).perform()

# 等待悬停后下拉列表出现
sleep(1)

# 在下拉列表中找到“天津”并点击
# 注意：这里使用 XPATH 或者 CSS 选择器，根据实际情况来
tianjin_city = driver.find_element(By.XPATH, "//div[@class='city20141104nr' and @id='cityi010']/a[text()='天津']")
tianjin_city.click()

# 切换到天津页面需要等待
sleep(2)


In [12]:
# 找到左侧区域列表中，“南开”对应的链接（注意这里的链接文字要和真实页面保持一致）
nankai_link = driver.find_element(By.LINK_TEXT, "南开")
nankai_link.click()

# 等待页面加载
sleep(2)


In [13]:
# 找到“八里台”对应的链接
balitai_link = driver.find_element(By.LINK_TEXT, "八里台")
balitai_link.click()

# 等待页面加载
sleep(2)


In [ ]:
from selenium.common.exceptions import NoSuchElementException

house_data = []  # 用于汇总所有页面的房源数据

while True:
    # ------------------ 1. 找到房源列表并解析当前页数据 ------------------
    try:
        shop_list_container = driver.find_element(By.CSS_SELECTOR, "div.shop_list.shop_list_4")
    except NoSuchElementException:
        print("未找到房源列表的主容器，请检查页面结构。")
        break  # 若连主容器都无法找到，直接退出循环
    
    # 找到每一条房源的dl标签
    dl_elements = shop_list_container.find_elements(By.TAG_NAME, "dl")

    for dl in dl_elements:
        try:
            # 标题信息
            title = dl.find_element(By.XPATH, "./dd/h4/a/span").text.strip()

            # 户型 & 面积
            house_info = dl.find_element(By.XPATH, "./dd/p[@class='tel_shop']").text.strip()
            # 例如 "2室2厅 | 109.07㎡ | 高层 （共9层） | 南北向 | 2006年建 | 中介姓名"

            # 价格
            price_total = dl.find_element(By.XPATH, "./dd[@class='price_right']/span[1]").text.strip()  # 总价
            price_unit  = dl.find_element(By.XPATH, "./dd[@class='price_right']/span[2]").text.strip()   # 单价，如 "34656元/㎡"

            # 存储数据到列表
            house_data.append({
                "标题": title,
                "户型_面积等": house_info,
                "总价": price_total,
                "单价": price_unit
            })
        except NoSuchElementException:
            # 有些dl可能是广告位或缺少字段，直接忽略
            continue

    # ------------------ 2. 找到“下一页”并点击翻页，如果找不到则结束 ------------------
    try:
        sleep(2)
        # “下一页”按钮常见几种定位方式：
        # 1) 文本为“下一页”的链接    -> By.LINK_TEXT, "下一页"
        # 2) CSS 选择器  p.last > a  -> By.CSS_SELECTOR, "p.last > a"
        # 根据你的页面实际情况来选择
        next_page = driver.find_element(By.LINK_TEXT, "下一页")
        next_page.click()
        # 等待 3 秒让页面加载
        sleep(3)
    except NoSuchElementException:
        print("没有找到下一页按钮, 爬取结束!")
        break

# 全部页面爬取结束后，将 house_data 转为 DataFrame
df = pd.DataFrame(house_data)
df


没有找到下一页按钮, 爬取结束!


,标题,户型_面积等,总价,单价
0,翠薇园 2室2厅 109.07平 电梯洋房 一梯两户 不到顶,2室2厅 | 109.07㎡ |高层 （共9层） | 南北向 | 2006年建 |宋宝静,378万,34656元/㎡
1,急售 水上公园旁楼层低采光好 低楼层 集体可以落户可上学,1室0厅 | 23.08㎡ |低层 （共6层） | 南向 | 2019年建 |邱红梅,43万,18630元/㎡
2,"佣.金1%保真 欣苑公寓一楼房子过五年,双卫,看房方便有钥匙",2室1厅 | 113.48㎡ | 底层 （共24层） | 东北向 | 2000年建 |崔亮,205万,18064元/㎡
3,佣.金1% 保真 公安局家属院 房子少税费 南北通透楼层好,2室1厅 | 81.52㎡ |高层 （共7层） | 南北向 | 1999年建 |崔亮,202万,24779元/㎡
4,佣.金1% 保真 观园公寓5期5楼啊 位置好 户型好 满五年,3室1厅 | 124.12㎡ |高层 （共7层） | 南北向 | 2002年建 |崔亮,290万,23364元/㎡
...,...,...,...,...
1062,"拥金1,南开区红旗南路附近中楼层南北通透两室采光好小区环境好",2室1厅 | 82.65㎡ |中层 （共7层） | 南北向 | 1997年建 |靳雪辉,220万,26618元/㎡
1063,佣.金1% 南开红旗南路水上公园旁中楼层双地铁沿线 好位置,2室1厅 | 75.63㎡ |中层 （共6层） | 南北向 | 1997年建 |靳雪辉,205万,27105元/㎡
1064,居祥南里 水上公园 企业产 交通便利 诚意出售,1室1厅 | 24.92㎡ |中层 （共6层） | 南向 | 2020年建 |安呈勇,65万,26083元/㎡
1065,"拥金1,南开区南片三楼靠近公园环境好 嘉泰花园 2室1厅 9",2室1厅 | 95.92㎡ |中层 （共7层） | 南北向 | 2000年建 |靳雪辉,220万,22935元/㎡


In [17]:
import pandas as pd
import re

# 假设 df 是之前爬取到的数据
df_cleaned = df.copy()

# 1. 提取平方数（㎡）：从 "户型_面积等" 中匹配第一个 "|" 和第二个 "|" 之间的内容
def extract_square_meters(house_info):
    """
    利用正则表达式从字符串中提取面积，
    匹配类似 "2室2厅 | 109.07㎡ | ..." 的格式，
    返回浮点数面积，去除单位 "㎡"
    """
    try:
        # 此处匹配 | 后面空格、数字（包括小数点）和"㎡"的模式
        match = re.search(r"\|\s*([\d.]+)㎡\s*\|", house_info)
        return float(match.group(1)) if match else None
    except Exception as e:
        return None

df_cleaned["平方数（㎡）"] = df_cleaned["户型_面积等"].apply(extract_square_meters)

# 2. 处理总价（万），去掉 "万" 单位
df_cleaned["总价（万）"] = df_cleaned["总价"].str.replace("万", "", regex=False).astype(float)

# 3. 处理单价（元/㎡），去掉 "元/㎡" 单位
df_cleaned["单价（元/㎡）"] = df_cleaned["单价"].str.replace("元/㎡", "", regex=False).astype(float)

# 4. 仅保留需要的三列
df_cleaned = df_cleaned[["平方数（㎡）", "总价（万）", "单价（元/㎡）"]]

# 将清洗后的 DataFrame 存储到 CSV 文件，不保存索引
df_cleaned.to_csv("house_data_cleaned.csv", index=False)

print("数据已保存到 house_data_cleaned.csv")

数据已保存到 house_data_cleaned.csv
